In [1]:
import nibabel as nbl
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import os
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pickle import dump
from sklearn.model_selection import train_test_split
from loguru import logger
import joblib
from patient_list import get_test_patient_ids

In [2]:
pwd

'/Users/neginpiran/PycharmProjects/image_quality/data'

In [3]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
patient_test_list = get_test_patient_ids()
patient_test_list

{1,
 4,
 8,
 12,
 15,
 19,
 23,
 27,
 32,
 33,
 34,
 37,
 39,
 48,
 59,
 61,
 65,
 66,
 77,
 100,
 102,
 111,
 117,
 120,
 124,
 132,
 144,
 152,
 153,
 168}

In [4]:
pwd

'/Users/neginpiran/PycharmProjects/image_quality/data'

In [5]:
csv_path = "/Users/neginpiran/OneDrive/Documents/ImageQuality/all_labels.xlsx" # T2 path
# csv_path = "/Users/neginpiran/OneDrive/Documents/ImageQuality/img_quality_adc.xlsx" # ADC path
df = pd.read_excel(csv_path)
df.head()

,Study ID,MRN,Date of MRI,Rectal Diameter; AP direction in cm,"Rectal Content 0-collapsed rectum, no significant rectal content; 1-mostly solid stool, 2-mostly liquid stool, 3-mostly gas",label,"Overall image quality DWI 1=non-diagnostic, 2=below average image quality but remains diagnostic, 3=average image quality, 4=above average image quality and 5=excellent image quality"
0,1,9851643,2019-01-03,6.2,3,2,4
1,2,41018870,2019-01-14,3.8,3,1,4
2,3,22335004,2019-01-16,2.2,3,2,5
3,4,24459091,2019-01-25,6.8,3,1,3
4,5,34866467,2019-01-23,4.5,3,1,4


In [6]:
rootdir = '/Users/neginpiran/OneDrive/Documents/ImageQuality/Prostate Quality Unsegmented Cases'

img_list, label_list = [], []
counter = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        counter += 1
        # # T2 and not ADC nor loc
        if 'DS_Store' not in file and 'DICOMDIR' not in file and "T2" in os.path.join(subdir, file) and 'ADC' not in os.path.join(subdir, file) and 'LOC' not in os.path.join(subdir, file):
        # ADC and not T2 nor loc
        # if 'DS_Store' not in file and 'DICOMDIR' not in file and "ADC" in os.path.join(subdir, file) and 'T2' not in os.path.join(subdir, file) and 'LOC' not in os.path.join(subdir, file):
            # print(f'------- {subdir}')
            # print(f'------- {subdir}')
            image_path = os.path.join(subdir, file)
            start_index = subdir.lower().find('ai study id')+107
            end_index = -2 # -2 for T2 and -3 for ADC
            # print(f"subdir: {subdir}")
            # print(f'start_index: {start_index}')
            # print(f"ID: {subdir[start_index:end_index]}")
            ID = subdir[start_index:end_index]
            if int(ID) not in patient_test_list:
                # print(f'ID: {ID}')
                dc_ar = dicom.dcmread(image_path).pixel_array
                label = df[df['Study ID']==int(ID)]['label'].values[0]
                img_list.append(dc_ar)
                label_list.append(label)


In [7]:
# resize all images
X = np.zeros((len(img_list), IMG_HEIGHT, IMG_WIDTH))
y = np.zeros(len(img_list))
for i in range(len(img_list)):
    img = resize(img_list[i], (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X[i,:,:] = img
    y[i] = label_list[i]
    

In [8]:
len(img_list)

3820

In [9]:
y.max()

2.0

In [10]:
scaler = MinMaxScaler()

# X_all = np.concatenate((X_peripheral, X_anterior), axis=0)
# y_all = np.concatenate((y_peripheral, y_anterior), axis=0)
X_scld = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
joblib.dump(scaler, "./scaler_patient.joblib")
X_final = np.expand_dims(X_scld, axis=3)
y_final = np.expand_dims(y, axis=1)


In [11]:
print(f'X_final.shape: {X_final.shape}')
print(f'y_final.shape: {y_final.shape}')

X_final.shape: (3820, 256, 256, 1)
y_final.shape: (3820, 1)


In [12]:
X_final.max()

1.0

In [13]:
X_final.mean()

0.19664417476820184

In [14]:
np.save('X_quality_train', X_final)
np.save('y_quality_train', y_final)